# Cutout Size Determination by detector

Cutout size varies with max dy, which varies by detector. This notebook can read a conf file and determine the max dy. Specifically, the scipy.optimize function minimize returns the minimum -dy (negative dy) and it's location (the negative of the minimum -dy is the maximum). The absolute value of this is the minimum size cutout.

1) Put the conf file in.
2) Run the notebook.
3) Note the value returned at 'x:'; this is the location of the maximum.
4) Note the value returned at 'func:'; this times -1 is the maximum value.
5) Test somewhere that setting size equal to this func value works, or one greater. This test is in case scipy got stuck in a local max/min.
6) Track the detector value.

det1: 77  - tested \
det2: 80 \
det3: 88 \
det4: 180 \
det5: 195 \
det6: 209 \
det7: 279 \
det8: 302 \
det9: 328 \
det10: 1 - use compute_size (object based sizing)? \
det11: 2 \
det12: 6 \
det13: 30 \
det14: 18 \
det15: 2 \
det16: 78 \
det17: 49 \
det18: 22

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, Bounds
from collections import OrderedDict

import os
os.chdir("/Users/keith/astr/research_astr/roman_configs")

In [2]:
config_file = lambda ii: "Roman.det%i.07242020.conf" %ii

def read_conf(ii):
    config = config_file(ii)
    conf = OrderedDict()
    lines = open(config).readlines()

    for line in lines:
        if (line.startswith('#')) | (line.strip() == '') | ('"' in line):
                    continue

        if line.startswith("DYDX_A_"):
            spl = line.split(';')[0].split('#')[0].split()
            param = spl[0]
            value = np.cast[float](spl[1:])
            conf[param] = value

    b = {}
    for n, dydx in enumerate(conf.keys()):
        for m, coef in enumerate(conf[dydx]):
            b[f"{n},{m}"] = coef
    
    return b

In [3]:
def function(b):
    a_0 = lambda i, j: b["0,0"] + b["0,1"]*i + b["0,2"]*j + b["0,3"]*i**2 + b["0,4"]*i*j + b["0,5"]*j**2
    a_1 = lambda i, j: b["1,0"] + b["1,1"]*i + b["1,2"]*j + b["1,3"]*i**2 + b["1,4"]*i*j + b["1,5"]*j**2
    a_2 = lambda i, j: b["2,0"] + b["2,1"]*i + b["2,2"]*j + b["2,3"]*i**2 + b["2,4"]*i*j + b["2,5"]*j**2
    dy = lambda args: a_0(args[0],args[1]) + a_1(args[0],args[1])*args[2] + a_2(args[0],args[1])*args[2]**2
    dy_negated = lambda args: -(a_0(args[0],args[1]) + a_1(args[0],args[1])*args[2] + a_2(args[0],args[1])*args[2]**2)

    function_dict = {"dy": dy, "dy_negated": dy_negated}

    return function_dict

In [4]:
def find_size(function_dict):
    bounds = Bounds([0,0,-800],[4800,4800,800])
    minimum = minimize(function_dict["dy"], x0=[4800,4800,800], bounds=bounds).fun
    maximum = minimize(function_dict["dy_negated"], x0=[4800,4800,800], bounds=bounds).fun

    rec_size = int(max(abs(minimum), abs(maximum)) + 1)

    return rec_size

In [5]:
def main(ii, q):
    b = read_conf(ii)
    function_dict = function(b)
    rec_size = find_size(function_dict)

    q.put(rec_size)
    
    return None

In [6]:
import multiprocess

In [7]:
def run():
    q = multiprocess.Queue()
    processes = [multiprocess.Process(target=main, args=(ii, q)) 
             for ii in range(1,19)]

    for process in processes:
        process.start()

    for process in processes:
        process.join()

    for ii in range(1,19):
        rec_size = q.get()
        print("Detector %i size rec: %i" %(ii, rec_size))

    return None

In [8]:
run()

Detector 1 size rec: 77
Detector 2 size rec: 80
Detector 3 size rec: 88
Detector 4 size rec: 180
Detector 5 size rec: 195
Detector 6 size rec: 209
Detector 7 size rec: 279
Detector 8 size rec: 302
Detector 9 size rec: 328
Detector 10 size rec: 98
Detector 11 size rec: 101
Detector 12 size rec: 110
Detector 13 size rec: 203
Detector 14 size rec: 217
Detector 15 size rec: 232
Detector 16 size rec: 303
Detector 17 size rec: 326
Detector 18 size rec: 353


In [9]:
def main_2(ii):
    b = read_conf(ii)
    function_dict = function(b)
    rec_size = find_size(function_dict)
    
    return rec_size

In [10]:
for ii in range(1,19):
    rec_size = main_2(ii, )
    print("Detector %i size rec: %i" %(ii, rec_size))

Detector 1 size rec: 77
Detector 2 size rec: 80
Detector 3 size rec: 88
Detector 4 size rec: 180
Detector 5 size rec: 195
Detector 6 size rec: 209
Detector 7 size rec: 279
Detector 8 size rec: 302
Detector 9 size rec: 328
Detector 10 size rec: 98
Detector 11 size rec: 101
Detector 12 size rec: 110
Detector 13 size rec: 203
Detector 14 size rec: 217
Detector 15 size rec: 232
Detector 16 size rec: 303
Detector 17 size rec: 326
Detector 18 size rec: 353
